In [9]:
PATH_SAIDA = "output/"

In [18]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [14]:
def mostrar_imagem(img, titulo="Imagem"):
    """
    Mostra a imagem em um notebook, convertendo de BGR -> RGB.
    """
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(titulo)
    plt.axis("off")
    plt.show()

In [3]:
def alterar_brilho(img, valor):
    """
    Altera o brilho somando 'valor' a cada pixel.
    """
    # converte para int16 para evitar overflow
    img_temp = img.astype(np.int16)  
    img_temp = img_temp + valor      # soma escalar em todos os canais
    img_temp = np.clip(img_temp, 0, 255)  # mantém no intervalo válido
    return img_temp.astype(np.uint8)

In [4]:
def negativo(img):
    """
    Gera o negativo da imagem (255 - pixel).
    """
    return 255 - img

In [11]:
img = cv2.imread("img.jpg")
img_brilho = alterar_brilho(img, 50)  # aumenta brilho em +50
cv2.imwrite(PATH_SAIDA+"saida_brilho.jpg", img_brilho)

True

In [12]:
img_negativo = negativo(img)
cv2.imwrite(PATH_SAIDA+"saida_negativa.jpg", img_negativo)

True

In [13]:
cv2.imshow("Resultado", img_brilho)
cv2.waitKey(0)

-1